In [7]:
from binance.client import Client

api_key = 'CRejMTuSYokbz2JeNBBo1cb1nt3IOL8d1pcAjeBoq9qavYKuqAQLWfxFMFsqn70o'
api_secret = 'OEKkQ2R328yMF4LAGw5K15tBwIuISYzjLpZ7nSQK8qHLWMI9AjGas6t31xvGDhUy'

client = Client(api_key, api_secret)

prices = client.get_all_tickers()

len(prices)

233

In [105]:
import urllib.request
import json
response = urllib.request.urlopen('https://bittrex.com/api/v1.1/public/getmarketsummaries')
summary=json.loads(response.read())


In [32]:
from datetime import datetime

class InvalidBase(Exception):
    pass

def convert_price(display_price,market_name):
    btc_price = 15010.00
    eth_price = 968.50
    
    base = market_name[:3]
    
    if base == 'BTC':
        return display_price*btc_price
    elif base == 'ETH':
        return display_price*eth_price
    elif base == 'USD':
        return display_price
    else:
        raise InvalidBase

date_threshold = '2017-06-30'
price_threshold_in_dollar = 1/6.5

filtered = [(record['MarketName'],convert_price(record['Ask'],record['MarketName']),record['Created'][:10]) for record in summary['result'] \
            if convert_price(record['Ask'],record['MarketName'])<price_threshold_in_dollar and \
            datetime.strptime(record['Created'][:10], '%Y-%m-%d')<datetime.strptime(date_threshold, '%Y-%m-%d')]

for i in filtered:
    print(i[0]+','+str(i[1])+','+i[2])

BTC-2GIVE,0.0247665,2016-05-16
BTC-ABY,0.0250667,2014-10-31
BTC-ARDR,0.0,2016-10-13
BTC-BITB,0.0202635,2015-02-15
BTC-BURST,0.0741494,2014-09-16
BTC-COVAL,0.0358739,2015-01-22
BTC-DGB,0.1055203,2014-08-20
BTC-DOGE,0.009606400000000001,2014-02-13
BTC-FLDC,0.064543,2015-02-23
BTC-MUSIC,0.0603402,2017-03-27
BTC-PINK,0.06559369999999999,2014-07-29
BTC-PTC,0.13959300000000002,2014-03-27
BTC-RDD,0.0205637,2014-02-25
BTC-SC,0.042478300000000004,2017-05-22
BTC-SWIFT,0.0,2014-10-03
BTC-XDN,0.032121399999999994,2014-06-21
BTC-XMY,0.0229653,2014-03-15


In [121]:
import time
from datetime import timedelta

total_btc=0
symbol='POEBTC'

utc_time = datetime.strptime("2017-11-20T02",
                             "%Y-%m-%dT%H")

base_time=(utc_time - datetime(1970, 1, 1)) // timedelta(milliseconds=1)




for hour_i in range(1):
    #start_time=base_time +hour_i*3600*1000
    start_time=1513454400000
    #end_time = start_time+3600*1000-1
    end_time = 1513457999999

    trades = client.get_aggregate_trades(symbol=symbol,startTime=start_time,endTime=end_time)
    hourly_btc=0
    trade_count=0
    for record in trades:
        total_btc += float(record['q'])*float(record['p'])
        hourly_btc +=  float(record['q'])*float(record['p'])
        trade_count +=1
    print("hour {}, hourly_btc {}".format(hour_i+8,hourly_btc))

print("total btc {}, total trades {}".format(total_btc,trade_count))

hour 8, hourly_btc 6.926325010000001
total btc 6.926325010000001, total trades 356


In [193]:
end_time=int(time.time()*1000)
start_time=end_time-2*3600*1000+1

#candles = client.get_klines(symbol='POEBTC', interval='1d',startTime=start_time,endTime=end_time)

prices = client.get_all_tickers()

#candles = client.get_klines(symbol='POEBTC', interval='1d')

price_ratio = 3

def get_base_price():
    base_price={}
    for base_symbol in ['BNBUSDT', 'BTCUSDT', 'ETHUSDT']:
        base_price[base_symbol]={}
        for record in client.get_klines(symbol=base_symbol, interval='1d'):
            base_price[base_symbol][record[0]] = (float(record[1]) + float(record[4]))/2
    return base_price



In [195]:
def convert_price_daily(price,symbol,time_stamp, base_p=get_base_price()):
    
    if symbol.endswith('BTC'):
        if base_p['BTCUSDT'].get(time_stamp,None) == None:
            return None
        return price*base_p['BTCUSDT'].get(time_stamp,None)
    elif symbol.endswith('ETH'):
        if base_p['ETHUSDT'].get(time_stamp,None) == None:
            return None
        return price*base_p['ETHUSDT'].get(time_stamp,None)
    elif symbol.endswith('BNB'):
        if base_p['BNBUSDT'].get(time_stamp,None) == None:
            return None
        return price*base_p['BNBUSDT'].get(time_stamp,None)
    elif symbol.endswith('USDT'):
        return price
    else:
        return None

print(base_price['ETHUSDT'][int(time.time()/3600/24)*3600*24*1000])


1009.08


In [180]:
from datetime import date

for record in prices:
    symbol=record['symbol']
    current_price=convert_price_daily(float(record['price']),symbol,int(time.time()/3600/24)*3600*24*1000)
    if current_price == None:
        continue
    
    min_price=10000000.0
    min_price_date=''
    for daily in client.get_klines(symbol=symbol, interval='1d'):
        daily_price = convert_price_daily(float(daily[3]),symbol,daily[0])
        if float(daily[3])>0 and daily_price!=None and daily_price<min_price:
            min_price = daily_price
            min_price_date = date.fromtimestamp(daily[0]/1000).strftime('%Y-%m-%d')
    if current_price < min_price * price_ratio:
       print('symbol {}, min price in USDT {}, min price date {}, current price in USDT {}'.format(symbol,min_price,min_price_date,current_price))

symbol MTLBTC, min price in USDT 3.1768821500000004, min price date 2017-10-08, current price in USDT 7.11602364
symbol MTLETH, min price in USDT 3.3457599999999994, min price date 2017-10-08, current price in USDT 7.052460120000001
symbol BTGBTC, min price in USDT 108.16605675, min price date 2017-10-23, current price in USDT 252.27835962
symbol BTGETH, min price in USDT 111.75424055, min price date 2017-10-27, current price in USDT 254.83709952
symbol BCDBTC, min price in USDT 28.159293, min price date 2017-12-07, current price in USDT 43.411149
symbol BCDETH, min price in USDT 29.1588896, min price date 2017-12-11, current price in USDT 46.720404
symbol DGDBTC, min price in USDT 72.285911, min price date 2017-11-26, current price in USDT 187.3488999
symbol DGDETH, min price in USDT 71.838223, min price date 2017-11-26, current price in USDT 186.7907988
symbol LTCETH, min price in USDT 192.08807499999998, min price date 2018-01-03, current price in USDT 279.07116479999996
symbol LTCU

In [189]:
import requests

r=requests.get('https://api.huobi.pro/v1/common/symbols')
huobi_symbols=r.json()['data']

for record in huobi_symbols:
    symbol = record['base-currency']+record['quote-currency']
    current_price = convert_price_daily(requests.get('https://api.huobi.pro/market/trade?symbol={}'.format(symbol)).\
        json['tick']['data'][0]['price'])
    for dai

103


In [196]:
from my_agent import Agent


In [198]:
df = Agent('binance').get_all_klines()

getting kline for all symbols...


100%|████████████████████████████████████| 241/241 [01:39<00:00,  2.42symbol/s]


In [200]:
df2=df.reset_index()

df2[df2.index.duplicated()]

,index,open_ts,open,high,low,close,volume,asset_volume,trades,symbol,interval


In [201]:
df_base = (df2.assign(rn=df2.sort_values(['open_ts'], ascending=True).groupby(['symbol']).cumcount() + 1).query('rn == 1'))

In [250]:
import pandas as pd
import numpy as np
days = 7
grow_ratio = 2.0
pd.merge(df2,df_base, on='symbol').query('open_ts_x < open_ts_y +{} and open_ts_x > open_ts_y'.format(days*24*3600*1000))\
    .groupby('symbol').agg({'high_x':np.max,'high_y':np.max}).astype(dtype='float64').query('high_x>high_y*{}'.format(grow_ratio))

,high_x,high_y
symbol,,
BNBETH,0.004876,0.000888
ETCETH,1.000000,0.040000
GASBTC,0.009499,0.003100
GTOBNB,1.000000,0.450000
LENDBTC,0.000005,0.000003
MCOBTC,0.006247,0.002890
POWRBTC,0.000033,0.000013
POWRETH,0.000789,0.000270
SNGLSETH,0.011611,0.000880


In [251]:
df_huobi=Agent('huobi').get_all_klines()

getting all symbol and price for huobi...


100%|██████████████████████████████| 123/123 [02:50<00:00,  1.39s/huobi_symbol]


getting kline for all symbols...


100%|████████████████████████████████████| 121/121 [02:22<00:00,  1.17s/symbol]


In [254]:
df2.head()

,index,open_ts,open,high,low,close,volume,asset_volume,trades,symbol,interval
0,0,1515715200000,0.90000000,99.00000000,0.10000000,0.17116000,467088.33000000,112757.91587470,6983,RLCBNB,1d
1,1,1515801600000,0.16920000,0.17230000,0.16000000,0.16688000,17354.53000000,2844.04782810,254,RLCBNB,1d
2,0,1515715200000,0.00250000,0.01890000,0.00102000,0.00300600,3583116.57000000,15689.16519315,41218,RLCETH,1d
3,1,1515801600000,0.00300100,0.00300600,0.00280900,0.00297600,90821.70000000,264.36341551,1176,RLCETH,1d
4,0,1515715200000,0.00037000,0.00109000,0.00026000,0.00027440,16168061.14000000,6229.73023419,104632,RLCBTC,1d


In [268]:
agent_binance=Agent('binance')
df_poe = agent_binance.get_symbol_kline(symbol='MAMAETH',interval='1h',limit=13)
print(df_poe[['asset_volume','symbol']])

BinanceAPIException: APIError(code=-1121): Invalid symbol.

In [269]:
df_base

,index,open_ts,open,high,low,close,volume,asset_volume,trades,symbol,interval,rn
0,0,1515715200000,0.90000000,99.00000000,0.10000000,0.17116000,467088.33000000,112757.91587470,6983,RLCBNB,1d,1
2,0,1515715200000,0.00250000,0.01890000,0.00102000,0.00300600,3583116.57000000,15689.16519315,41218,RLCETH,1d,1
4,0,1515715200000,0.00037000,0.00109000,0.00026000,0.00027440,16168061.14000000,6229.73023419,104632,RLCBTC,1d,1
6,0,1515542400000,0.00120000,0.00795700,0.00116000,0.00189780,38817273.00000000,80797.47887650,149379,VIBEETH,1d,1
10,0,1515542400000,0.00009620,0.00070000,0.00005700,0.00015877,171312708.23925391,32567.66832583,372068,VIBEBTC,1d,1
14,0,1515110400000,0.20000000,0.61098000,0.06000000,0.12994000,406791.20000000,49560.25983390,1592,APPCBNB,1d,1
23,0,1515110400000,0.00860830,0.00860830,0.00105020,0.00192000,9563743.00000000,15537.35382190,17710,APPCETH,1d,1
32,0,1515110400000,0.00011999,0.00047638,0.00006540,0.00010726,17676992.00000000,1784.75379149,30781,APPCBTC,1d,1
41,0,1514505600000,50.00000000,50.00000000,0.50000000,0.86000000,75554.45000000,61979.31205640,2566,TRIGBNB,1d,1
57,0,1514505600000,0.01799000,0.90000000,0.00125800,0.00994400,328272.47000000,3172.43414674,8811,TRIGETH,1d,1


In [279]:
df_huobi=Agent('huobi').get_all_klines()

getting all symbol and price for huobi...


100%|██████████████████████████████| 131/131 [03:04<00:00,  1.41s/huobi_symbol]


getting kline for all symbols...


100%|████████████████████████████████████| 129/129 [02:45<00:00,  1.28s/symbol]


In [281]:
df_huobi[df_huobi['symbol']=='dbceth']

,volume,close,trades,high,open_ts,low,open,asset_volume,symbol,interval
0,4.851426e+06,0.000155,2093,0.000171,1516032000,0.000147,0.000147,791.812407,dbceth,1d
1,1.306903e+07,0.000147,5633,0.000204,1515945600,0.000120,0.000178,2362.861856,dbceth,1d
2,1.278940e+07,0.000178,6098,0.000224,1515859200,0.000170,0.000214,2452.530386,dbceth,1d
3,1.274334e+07,0.000214,5764,0.000290,1515772800,0.000160,0.000279,3120.033347,dbceth,1d
4,5.143945e+06,0.000279,2944,0.000410,1515686400,0.000247,0.000247,1495.966284,dbceth,1d


In [294]:
%load_ext autoreload

%autoreload 2


In [297]:
Agent('huobi').get_symbol_kline(symbol='dbceth',limit=500,interval='1d')

,volume,close,trades,high,open_ts,low,open,asset_volume,symbol,interval
0,5.365346e+06,0.000147,2313,0.000171,1516032000000,0.000145,0.000147,868.918849,dbceth,1d
1,1.306903e+07,0.000147,5633,0.000204,1515945600000,0.000120,0.000178,2362.861856,dbceth,1d
2,1.278940e+07,0.000178,6098,0.000224,1515859200000,0.000170,0.000214,2452.530386,dbceth,1d
3,1.274334e+07,0.000214,5764,0.000290,1515772800000,0.000160,0.000279,3120.033347,dbceth,1d
4,5.143945e+06,0.000279,2944,0.000410,1515686400000,0.000247,0.000247,1495.966284,dbceth,1d
